In [1]:
# HACK: use project root as the working directory 
from pathlib import Path

while Path.cwd().name != 'language-model-toxicity':
    %cd ..

/home/sam/research/language-model-toxicity


In [ ]:
from utils.constants import DATA_DIR
from pathlib import Path
from tqdm.auto import tqdm
import logging
import numpy as np

logging.disable(logging.CRITICAL)  # Disable logging from transformers

## Load WebText

In [9]:
# Path to webtext subsets:
webtext_dir = DATA_DIR / 'webtext'

webtext_files = sorted([file for file in webtext_dir.iterdir() if file.suffix == '.npy'])
webtext_files

[PosixPath('/data/language-model-toxicity/data/webtext/x00_tokens.npy'),
 PosixPath('/data/language-model-toxicity/data/webtext/x01_tokens.npy'),
 PosixPath('/data/language-model-toxicity/data/webtext/x02_tokens.npy'),
 PosixPath('/data/language-model-toxicity/data/webtext/x03_tokens.npy'),
 PosixPath('/data/language-model-toxicity/data/webtext/x04_tokens.npy'),
 PosixPath('/data/language-model-toxicity/data/webtext/x05_tokens.npy'),
 PosixPath('/data/language-model-toxicity/data/webtext/x06_tokens.npy'),
 PosixPath('/data/language-model-toxicity/data/webtext/x07_tokens.npy'),
 PosixPath('/data/language-model-toxicity/data/webtext/x08_tokens.npy'),
 PosixPath('/data/language-model-toxicity/data/webtext/x09_tokens.npy'),
 PosixPath('/data/language-model-toxicity/data/webtext/x10_tokens.npy'),
 PosixPath('/data/language-model-toxicity/data/webtext/x11_tokens.npy'),
 PosixPath('/data/language-model-toxicity/data/webtext/x12_tokens.npy'),
 PosixPath('/data/language-model-toxicity/data/webt

In [42]:
import numpy as np
from typing import List

EOS = 50256

def split_docs(tokens: np.array) -> List[np.array]:
    idx = np.nonzero(tokens == EOS)[0]
    docs = np.split(tokens, idx)
    docs = [doc[1:] for doc in docs if len(doc) > 1]
    return docs

In [45]:
# Try loading one document from subset of WebText
wt0 = np.load(webtext_files[0])
wt0docs = split_docs(webtext_0)
wt0docs[0]

array([  464,  1578,  4498, 24880,   468,  1775,  1865,  1194,  1700,
          12, 13395,  6591,  1176, 36427,  8406,    13, 13098, 43941,
        2722,   262,  9016,    12,   964,  8406,   329,   257,  6591,
       31392,  1628,   379,   257, 14702,   362,    13,  3682, 44359,
          14,    74,  1199,    11,  2263,   736,   262,  3670,   286,
       28918,  6591,  1176,  1628,   422, 17456,    13,   198,   198,
         464,  1700,    12, 13395,  8406,   468,   587,  8948,   416,
         257, 36451,   286,  3999,  8265, 11554,   449,   676, 34049,
        6192,   290,  4960,  8517,  1526,   549, 43850,    13,   198,
         198,  4826,    84, 43941, 42048,   290,  5638, 11416,  2722,
         257,  2472,   286,   718, 27837,   329,   262,  5150, 13803,
       29961,  6591, 31392,  1628,  6027,   284,   307,  3170,   287,
         262,  3240,   286,  2451,  1872,  7637,    11, 13098, 43941,
          13,  3806,   286,   718, 27837,    11,   262,  9016,  1683,
        8406,   286,

In [56]:
total_docs = 0

for file in tqdm(webtext_files):
    tokens = np.load(file)
    docs = split_docs(tokens)
    total_docs += len(docs)

total_docs

8282020

## Tokenize OpenWebText

In [48]:
from datasets.openwebtext_dataset import openwebtext_dataloader

owtc = openwebtext_dataloader(tokenized=True)

Loading list of OpenWebText files...


In [ ]:
with open('filenames.txt', 'a') as ff, open('tokens.dat', 'ab') as tf:
    for filename, tokens in tqdm(owtc):
        print(filename, file=ff)
        np.savetxt(tf, tokens)